# Instituto de Educação Superior de Brası́lia – IESB
## Pós-Graduação em Inteligência Artificial
### Aprendizagem Supervisionada

### Atividade 2 - Regressão Logística

Implemente um modelo de classificação com Regressão Logística para tentar determinar se uma amostra célula humana é benígna ou malígna, baseado em algumas características dessa amostra. 


#### Decrição do Dataset


| Atributo    | Descrição                   |
| ----------- | --------------------------- |
| ID          | Clump thickness             |
| Clump       | Clump thickness             |
| UnifSize    | Uniformity of cell size     |
| UnifShape   | Uniformity of cell shape    |
| MargAdh     | Marginal adhesion           |
| SingEpiSize | Single epithelial cell size |
| BareNuc     | Bare nuclei                 |
| BlandChrom  | Bland chromatin             |
| NormNucl    | Normal nucleoli             |
| Mit         | Mitoses                     |
| Class       | Benign or malignant         |

<br>
<br>

Cada linha do dataset corresponde a uma amostra de um determinado paciente.
O atributo alvo ('Class') contém o diagnóstico de cada amostra, com a seguinte codificação:

| 'Class' (Valor)   | Descrição                   |
| ---------- | --------------------------- |
| 2          | Benígno            |
| 4          | Malígno            |

### Henrique Brandão 

## 1) Carregue o dataset

 * Verifique o formato do dataset
 * Confira se os dados estão integros, isto é, 
   se não há valores faltantes, impossíveis ou possíveis outliers

In [1]:
import pandas as pd
import numpy as np

from datass.dataframe.inspection import _isnull

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
CSV = 'cell_samples.csv'
CLASSES = {2: 'GOOD', 4: 'BAD'}

In [3]:
df = pd.read_csv(CSV)
df.shape

(699, 11)

In [4]:
df.columns

Index(['ID', 'Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize',
       'BareNuc', 'BlandChrom', 'NormNucl', 'Mit', 'Class'],
      dtype='object')

In [5]:
df.head()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [6]:
df.describe()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BlandChrom,NormNucl,Mit,Class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


## 2) Realize, se necessário, um pré-processamento nos dados

  * Verifique se as features e a variavel alvo estão em valores numéricos.
  * Se necessário, faça um escalonamento nas features de forma a garantir que elas estarão em uma mesma escala numérica.
  * Separe as variáveis descritivas e alvo em dois tensores em formato Numpy.Array (__X__ e __y__, por exemplo).

In [7]:
df['BareNuc'].describe()

count     699
unique     11
top         1
freq      402
Name: BareNuc, dtype: object

In [8]:
df['BareNuc'].value_counts()

1     402
10    132
5      30
2      30
3      28
8      21
4      19
?      16
9       9
7       8
6       4
Name: BareNuc, dtype: int64

In [9]:
df[df['ID'] == 1057013]

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
23,1057013,8,4,5,1,2,?,7,3,1,4


In [10]:
df['BareNuc'] = df['BareNuc'].apply(lambda x: np.NaN if x == '?' else x)

In [11]:
df[df['ID'] == 1057013]

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
23,1057013,8,4,5,1,2,NaN,7,3,1,4


In [12]:
df['BareNuc'].value_counts()

1     402
10    132
5      30
2      30
3      28
8      21
4      19
9       9
7       8
6       4
Name: BareNuc, dtype: int64

In [13]:
_isnull(df)

>> Null registers:

# ID: 0 null rows
# Clump: 0 null rows
# UnifSize: 0 null rows
# UnifShape: 0 null rows
# MargAdh: 0 null rows
# SingEpiSize: 0 null rows
# BareNuc: 16 rows
# BlandChrom: 0 null rows
# NormNucl: 0 null rows
# Mit: 0 null rows
# Class: 0 null rows


In [14]:
df.dropna(inplace=True)

In [15]:
df.head()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [16]:
df.shape

(683, 11)

In [17]:
df.describe()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BlandChrom,NormNucl,Mit,Class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [18]:
df['Class'].value_counts()

2    444
4    239
Name: Class, dtype: int64

## 3) Separe o conjunto de dados em Treino e Teste

- Divida o dataset em subconjuntos de treino e de teste.
- Utilize 80% para treinamento e 20% para teste.
- Garanta que, nos dois subconjuntos, a variavel alvo mantenha a mesma proporção original de todo o dataset (faça a subamostragem estratificada).



In [19]:
X = df.drop(axis=1, columns=['ID', 'Class'])
Y = df['Class']

X.shape, Y.shape

((683, 9), (683,))

In [20]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2,
                                                shuffle=True,
                                                random_state=6,
                                                stratify=Y)

xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((546, 9), (546,), (137, 9), (137,))

## 4) Instancie um modelo de Regressão Logística e treine-o com a base de treinamento

In [21]:
logreg = LogisticRegression(max_iter=1e2, verbose=1)

In [22]:
logreg.fit(xtrain, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(max_iter=100.0, verbose=1)

## 5) Após treinar o modelo, verifique seu desempenho
 - Faça inferência (predições) nos dois subconjuntos (treino e teste).
 - Com o resultado das inferências, apresente um relatório ('classification_report') conténdo algumas métricas de desempenho (acurácia, precisão, f1, etc...) para as duas bases.
 - Apresente uma matriz de confusão para as duas bases.

In [23]:
logreg.predict(xtest)

array([2, 4, 4, 4, 2, 2, 2, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2,
       2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 4, 2, 2, 4, 2,
       4, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 2, 4,
       2, 4, 4, 4, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4,
       4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2,
       2, 4, 4, 2, 2])

In [24]:
logreg.score(xtest, ytest)

0.9416058394160584

## 6) Analise o resultado
 - Após calcular as métricas de desempenho para as duas bases, analise o desempenho do seu modelo.
 - O modelo treinado está bem ajustado ou há indícios de overfitting?

In [25]:
print(confusion_matrix(y_true=ytest, y_pred=logreg.predict(xtest)))

[[85  4]
 [ 4 44]]


In [26]:
print(classification_report(y_true=ytest, y_pred=logreg.predict(xtest)))

              precision    recall  f1-score   support

           2       0.96      0.96      0.96        89
           4       0.92      0.92      0.92        48

    accuracy                           0.94       137
   macro avg       0.94      0.94      0.94       137
weighted avg       0.94      0.94      0.94       137



O modelo está bem ajustado.